In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn')

In [2]:
#импорт данных
data = pd.read_csv('_train_sem09.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Подготовка данных путем разделения на признаки и отдельный целевой признак
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [4]:
#разделяем данные на тестовые и тренеровочные выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 69, test_size = 0.2)

In [5]:
#Обучим модель при помощи логистической регресии, гипперпаратметры по умолчанию

#Создаём объект класса LinearRegression
log_reg_base_model = linear_model.LogisticRegression()
log_reg_base_model.fit(X_train, y_train)
y_test_predict = log_reg_base_model.predict(X_test)

#оценим метрику F1
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, y_test_predict)))

f1_score на тестовом наборе: 0.777


/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
#Обучим модель при помощи случайного леса, гипперпаратметры по умолчанию

rf_base_model = ensemble.RandomForestClassifier()
rf_base_model.fit(X_train, y_train)
y_pred = rf_base_model.predict(X_test)
#оценим метрику F1
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, y_test_predict)))

f1_score на тестовом наборе: 0.777


F1-score получилась одиноаковой для моделей обученных при помощи логистической регрессии и при помощи случайного леса. Гипперапараметры были выбраны по умолчнаию. 
Подберем гипперпараметры и сраним результаты.
Выполним подбор гипперпараметров для модели логистической регрессии методами GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna.
Начнем с GridSeachCV

# Подбор гиперпараметров для модели логистической регрессии методом GridSeachCV

In [7]:
#Вариант с малым колличеством комбинаций гиперпарамтеров
from sklearn.model_selection import GridSearchCV
param_grid = {'penalty': ['l2', 'elasticnet', 'none'] ,#тип регурялизации
              'solver': ['lbfgs', 'saga'] #алгоритм оптимизации
            }
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=69, #генератор случайных чисел
        max_iter=8 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)
grid_search.fit(X_train, y_train) 
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, y_test_pred)))


/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

Наилучшие значения параметров: {'penalty': 'l2', 'solver': 'saga'}
f1_score на тестовом наборе: 0.799


/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [8]:
#Вариант с большим колличеством комбинаций гиперпарамтеров
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регуляризации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=69, max_iter=8), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
 
# %time - замеряет время выполнения
%time 
grid_search_1.fit(X_train, y_train) 
y_test_pred = grid_search_1.predict(X_test)
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))


CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.77 µs


/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `p

f1_score на тестовом наборе: 0.787
Наилучшие значения гиперпараметров: {'C': 0.5, 'penalty': 'l1', 'solver': 'liblinear'}


/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### Промежуточный вывод
##### Был применен метод поиска гиперпараметров GridSerachCV на двух наборах гиперпараметров. В обоих случаях удалсь улучшить целевую метрику. Но вариант с меньшим набором гиперпараметров справился лучше. 

# Подбор гипперпараметров для модели случайного леса методом GridSeachCV

In [9]:
#Вариант с малым колличеством комбинаций гиперпарамтеров
param_grid = [
              {'max_depth': [5, 8, 10],
               'min_samples_leaf': [7, 8, 9] }, #первый словарь 
              {'n_estimators': [100, 200, 300], 
               'max_depth': [5, 8, 10] } #второй словарь 
             ]
grid_search = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(
        random_state=69, #генератор случайных чисел
        #max_iter=8 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)
grid_search.fit(X_train, y_train) 
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения параметров: {'max_depth': 10, 'n_estimators': 200}
f1_score на тестовом наборе: 0.818


#### Промежуточный вывод
##### Был применен метод поиска гиперпараметров GridSerachCV для поиска лучших гиперпараметров модели случайного леса. Метрику удалось улучшить относительно базовой модели и при подборе параметров аналогичным методотом для модели логистической регрессии.  

# Подбор гиперпараметров для модели логистической регрессии методом RandomSeachCV

In [10]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регуляризации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
random_search_forest = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=69), 
    param_distributions = param_grid,
    cv=5,
    n_jobs = -1
)  
random_search_forest.fit(X_train, y_train) 
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


f1_score на тестовом наборе: 0.798
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.1}


/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


# Подбор гиперпараметров для модели для модели случайного леса методом RandomSeachCV

In [11]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {'n_estimators': list(range(100, 200, 30)),
              'min_samples_leaf': [5,7],
              'max_depth': [10, 15, 20, 25]
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=69), 
    param_distributions = param_grid,
    cv=5,
    n_jobs = -1
)  
random_search_forest.fit(X_train, y_train) 
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

f1_score на тестовом наборе: 0.819
Наилучшие значения гиперпараметров: {'n_estimators': 130, 'min_samples_leaf': 5, 'max_depth': 25}


#### Промежуточный вывод
##### Был применен метод поиска гиперпараметров RandomSerachCV для поиска лучших гиперпараметров модели случайного леса. Метрику практически не удалось улучшить относительно метода GridSearchCV. Однако на данном этапе пожно сделать предположение о том, что на этом наборе данных лучше работает модель случайного леса. И в RandomSerachCV и в GridSearchCV F1 score получилась выше именно при использовании модели случаного леса.

# Подбор гиперпараметров для модели логистической регрессии методом Hyperopt

In [12]:
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

In [13]:
#задаем пространство гиперпараметров
search_space = {
                 'C' : hp.uniform('C', 0.05, 3),
            'solver' : hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear']),
                }

In [36]:
# напишем функцкцию для подсчета целевой метрики 
random_state = 69
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)
    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    return -score

In [15]:
 # напишем функцкцию для подсчета целевой метрики с кросс валидацией
random_state = 69
def hyperopt_rf_cv(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)
    # обучаем модель
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    return -score

In [35]:
# начинаем подбор гиперпараметров с функцией без сросс валидации


trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=search_space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))


{'C': 1.5937319011187387, 'solver': 'newton-cg'}      
{'C': 1.9715389617316297, 'solver': 'liblinear'}                                 
{'C': 1.6958113907702441, 'solver': 'liblinear'}                                 
{'C': 2.5675768151943754, 'solver': 'lbfgs'}                                     
{'C': 0.13374704064517903, 'solver': 'lbfgs'}                                    
 20%|██        | 4/20 [00:04<00:13,  1.16trial/s, best loss: -0.9003039513677812]

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



{'C': 1.5869466340465808, 'solver': 'lbfgs'}                                     
 25%|██▌       | 5/20 [00:05<00:10,  1.38trial/s, best loss: -0.9003039513677812]

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



{'C': 1.8498807134097373, 'solver': 'liblinear'}                                 
 30%|███       | 6/20 [00:05<00:09,  1.55trial/s, best loss: -0.9003039513677812]

...

{'C': 1.8623727388234486, 'solver': 'lbfgs'}                                      
{'C': 1.735928233395316, 'solver': 'lbfgs'}                                       
 85%|████████▌ | 17/20 [00:14<00:02,  1.20trial/s, best loss: -0.9003039513677812]

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



{'C': 1.0955142676920484, 'solver': 'liblinear'}                                  
 90%|█████████ | 18/20 [00:15<00:01,  1.38trial/s, best loss: -0.9003039513677812]

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



{'C': 2.898532819709438, 'solver': 'liblinear'}                                   
100%|██████████| 20/20 [00:16<00:00,  1.20trial/s, best loss: -0.9078144078144079]
Наилучшие значения гиперпараметров {'C': 2.898532819709438, 'solver': 2}


In [17]:
#получим словарь лучших параметров
from hyperopt import space_eval
best_param = space_eval(search_space, best)

In [18]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    random_state=random_state, 
    C=best_param['C'],
    solver=best_param['solver'])
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.780


In [19]:
#повоторим то де самое с кросс валидацией
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf_cv, # наша функция 
          space=search_space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

 15%|█▌        | 3/20 [00:13<00:58,  3.44s/trial, best loss: -0.771807095549252] 

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

 20%|██        | 4/20 [00:15<00:45,  2.86s/trial, best loss: -0.771807095549252]

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

 25%|██▌       | 5/20 [00:16<00:37,  2.52s/trial, best loss: -0.7862585853096332]

...

 80%|████████  | 16/20 [00:52<00:15,  3.94s/trial, best loss: -0.7862585853096332]

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

 85%|████████▌ | 17/20 [00:55<00:10,  3.40s/trial, best loss: -0.7862585853096332]

/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

100%|██████████| 20/20 [01:00<00:00,  3.01s/trial, best loss: -0.7862585853096332]
Наилучшие значения гиперпараметров {'C': 0.13374704064517903, 'solver': 1}


In [20]:
best_param = space_eval(search_space, best)


In [21]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    random_state=random_state, 
    C=best_param['C'],
    solver=best_param['solver'])
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.3f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.797


/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Промежуточный вывод
##### Был применен метод поиска гиперпараметров Hyperopt для поиска лучших гиперпараметров модели логистической регресии. Метрика чуть чуть улучшилась относительно RandomSearchСV, но все равно хуже RandomSearchСV для метода случайного леса.

# Подбор гиперпараметров для модели случайного леса методом Hyperopt

In [69]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

In [70]:
# напишем функцию для посчета целевой метрики 
random_state = 69
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    return -score

In [71]:
# напишем функцию для посчета целевой метрики с использование мкросс-валидации
random_state = 69
def hyperopt_rf_cv(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    return -score




In [72]:
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best_rf=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_rf))

#100%|██████████| 20/20 [01:10<00:00,  3.50s/it, best loss: -0.7986892215038526]
#Наилучшие значения гиперпараметров {'max_depth': 24.0, 'min_samples_leaf': 2.0, 'n_estimators': 153.0}
#CPU times: user 1min 10s, sys: 183 ms, total: 1min 10s
#Wall time: 1min 10s

100%|██████████| 20/20 [00:33<00:00,  1.69s/trial, best loss: -0.9917203311867526]
Наилучшие значения гиперпараметров {'max_depth': 23.0, 'min_samples_leaf': 2.0, 'n_estimators': 164.0}


In [73]:
#получим словарь лучших параметров
from hyperopt import space_eval
best_param = space_eval(space, best_rf)

In [74]:
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    max_depth=int(best_param['max_depth']),
    min_samples_leaf=int(best_param['min_samples_leaf']),
    n_estimators=int(best_param['n_estimators']))
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.8368098159509203


In [75]:
#повоторим то де самое с кросс валидацией
trials = Trials() # используется для логирования результатов

best_rf_cv=fmin(hyperopt_rf_cv, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_rf_cv))

100%|██████████| 20/20 [01:07<00:00,  3.39s/trial, best loss: -0.8108892689738496]
Наилучшие значения гиперпараметров {'max_depth': 23.0, 'min_samples_leaf': 2.0, 'n_estimators': 164.0}


In [76]:
best_param = space_eval(space, best_rf_cv)

In [77]:
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    max_depth=int(best_param['max_depth']),
    min_samples_leaf=int(best_param['min_samples_leaf']),
    n_estimators=int(best_param['n_estimators']))
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.8368098159509203


#### Промежуточный вывод
##### На данном этапе подбора гипперпараметров метод подбора HyperOpt для модели случайного леса показала самый лучший результат, при этом парметры парметры были подобраны одинковые при использваонии кросс-валидации и без нее.  

# Подбор гиперпараметров для модели логистической регрессии методом Optuna

In [125]:
import optuna

print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.1.0


In [127]:
#Настроим оптимизацию гиперпараметров для алгоритма логистической регресии без кросс-валидации.
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
  C = trial.suggest_float("C", 1e-2, 1)
  solver = trial.suggest_categorical('solver' , ['lbfgs','liblinear'])
  

  # создаем модель
  model = linear_model.LogisticRegression(C=C, solver = solver )
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score
  

In [128]:
#Настроим оптимизацию гиперпараметров для алгоритма логистической регресии c кросс-валидации.
def optuna_lr_cv(trial, cv=5):
  # задаем пространства поиска гиперпараметров
  C = trial.suggest_float("C", 1e-2, 1)
  solver = trial.suggest_categorical('solver' , ['lbfgs','liblinear'])
  

  # создаем модель
  model = linear_model.LogisticRegression(C=C, solver = solver )
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

  return score
  

In [129]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=20)

[I 2023-03-19 15:38:31,379] A new study created in memory with name: LogisticRegression
[I 2023-03-19 15:38:31,899] Trial 0 finished with value: 0.8843702579666161 and parameters: {'C': 0.815728013692765, 'solver': 'liblinear'}. Best is trial 0 with value: 0.8843702579666161.
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2023-03-19 15:38:32,430] Trial 1 finished with value: 0.8877395801642836 and parameters: {'C': 0.9487714528495543, 'solver': 'lbfgs'}. Best is trial 1 with value: 0.8877395801642836.
[I 2023

CPU times: user 8.72 s, sys: 264 ms, total: 8.99 s
Wall time: 8.99 s


In [130]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'C': 0.9804868087895373, 'solver': 'liblinear'}
f1_score на обучающем наборе: 0.89


In [131]:
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


In [132]:
#постоврим для алгоритма с кросс-валидацией

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr_cv, n_trials=20)

[I 2023-03-19 15:38:51,554] A new study created in memory with name: LogisticRegression
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ivan/anaconda3/envs/Ivan/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for

In [133]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'C': 0.021834429769237662, 'solver': 'lbfgs'}
f1_score на обучающем наборе: 0.79


In [134]:
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.80


#### Промежуточный вывод
##### Был проведен подбор гипперпараметров моедли линейной регрессии методом Optuna, результат удалось учлучшить, однако целевая метрика все равно ниже, чем при использовании модели случайного леса 

# Подбор гиперпараметров для модели случайного леса методом Optuna


In [135]:
#Настроим оптимизацию гиперпараметров для алгоритма случайного леса без кросс-валидации.
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score
  

In [137]:
#Настроим оптимизацию гиперпараметров для алгоритма случайного леса с кросс-валидации.
def optuna_rf_cv(trial, cv=5):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
  return score
  

In [140]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2023-03-19 15:49:43,184] A new study created in memory with name: RandomForestClassifier
[I 2023-03-19 15:49:47,769] Trial 0 finished with value: 0.8051958484644082 and parameters: {'n_estimators': 189, 'max_depth': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8051958484644082.
[I 2023-03-19 15:49:50,744] Trial 1 finished with value: 0.8116907314590861 and parameters: {'n_estimators': 104, 'max_depth': 14, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.8116907314590861.
[I 2023-03-19 15:49:54,518] Trial 2 finished with value: 0.8097766843772337 and parameters: {'n_estimators': 132, 'max_depth': 13, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.8116907314590861.
[I 2023-03-19 15:49:57,499] Trial 3 finished with value: 0.8074165128319744 and parameters: {'n_estimators': 117, 'max_depth': 15, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.8116907314590861.
[I 2023-03-19 15:50:00,196] Trial 4 finished with value: 0.8120270780485775 and parameters: {'n_

CPU times: user 27.4 s, sys: 739 ms, total: 28.1 s
Wall time: 1min 6s


In [143]:
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {}".format(study.best_value))

Наилучшие значения гиперпараметров {'n_estimators': 125, 'max_depth': 13, 'min_samples_leaf': 4}
f1_score на обучающем наборе: 0.8140569524096444


In [144]:
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.825242718446602


In [145]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf_cv, n_trials=20)

[I 2023-03-19 15:51:52,174] A new study created in memory with name: RandomForestClassifier
[I 2023-03-19 15:51:55,402] Trial 0 finished with value: 0.8020501761319917 and parameters: {'n_estimators': 147, 'max_depth': 12, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.8020501761319917.
[I 2023-03-19 15:51:59,557] Trial 1 finished with value: 0.80303175901725 and parameters: {'n_estimators': 194, 'max_depth': 10, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.80303175901725.
[I 2023-03-19 15:52:02,851] Trial 2 finished with value: 0.7973865527367072 and parameters: {'n_estimators': 134, 'max_depth': 27, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.80303175901725.
[I 2023-03-19 15:52:06,650] Trial 3 finished with value: 0.7994487868804585 and parameters: {'n_estimators': 158, 'max_depth': 23, 'min_samples_leaf': 9}. Best is trial 1 with value: 0.80303175901725.
[I 2023-03-19 15:52:10,130] Trial 4 finished with value: 0.8036307043563837 and parameters: {'n_estimato

CPU times: user 29.2 s, sys: 527 ms, total: 29.7 s
Wall time: 1min 8s


In [146]:
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {}".format(study.best_value))

Наилучшие значения гиперпараметров {'n_estimators': 150, 'max_depth': 24, 'min_samples_leaf': 4}
f1_score на обучающем наборе: 0.8132554620506219


In [147]:
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.8226600985221676


# Итоги
### Исходные данные были разделены тренеровчную и тестовую выборки. Для тренеровнчной выборки были построены модели логестической регресии и случайного леса. На тестовой выборке были получены метрики f1_score для моделей с гипперапарметрами по умолчанию. Далее для каждого алгоритма были применены следующие методы подбора гипперпараметров: GridSearchCV, RandomSearchCV, HyperOpt и Optuna. Последние два метода применялись для кажого алгоритма дважды с ипользованием кросс валидации и без. Для кадого случая была получена метрика F1_score. По полученным метрикам видно, что самым эффективным методом оказлался HyperOpt для модели случайного леса. Метрика 0.84. Так же в целом видно, что модели логистической регерссии показывают результат хуже, чем случайный лес.